<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2%D0%92%D0%92_NLP_HW_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 9 Гладышев ВВ

## Генерация текста

In [1]:
import numpy as np
import tensorflow as tf
import string
from tqdm import tqdm
import os

In [2]:
path_to_file = '/content/evgenyi_onegin.txt'

vocab = set()
vocab.add("\n")
vocab.add(".")
vocab.add(",")
with open(path_to_file, "r", encoding='utf-8', errors='ignore') as f:
    for line in tqdm(f):
        line = ''.join([ch for ch in list(line) if ch not in string.punctuation])
        words = line.lower().strip().split(" ")
        for word in words:
            vocab.add(word)

6505it [00:00, 61421.83it/s]


In [3]:
word2vec = {}
idx2word = np.array(list(vocab))

for i, word in enumerate(vocab):
    word2vec[word] = i

In [4]:
word2vec["\n"], word2vec["."], idx2word[word2vec["."]]

(4630, 4024, '.')

In [5]:
text_as_int = []
with open(path_to_file, "r", encoding='utf-8', errors='ignore') as f:
    for line in tqdm(f):
        words = line.lower().strip().split(" ")
        for word in words:
            point, comma = False, False
            if "," in word:
                comma = True
            if "." in word:
                point = True
            word = ''.join([ch for ch in list(word) if ch not in string.punctuation])
            text_as_int.append(word2vec[word])
            if comma:
                text_as_int.append(word2vec[","])
            if point:
                text_as_int.append(word2vec["."])
        text_as_int.append(word2vec["\n"])

6505it [00:00, 73917.20it/s]


In [6]:
text_as_int[:15]

[4713,
 196,
 524,
 4630,
 0,
 4630,
 3764,
 1543,
 4630,
 929,
 7366,
 8000,
 4630,
 0,
 4630]

In [7]:
len(text_as_int)

35895

### train and target

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text_as_int)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(25):
    print(idx2word[i.numpy()])

александр
сергеевич
пушкин





евгений
онегин


роман
в
стихах





не
мысля
гордый
свет
забавить
,


вниманье
дружбы
возлюбя


In [9]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)
sequences

<_BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int32, name=None)>

In [10]:
for item in sequences.take(3):
    print(' '.join(idx2word[item.numpy()]))

александр сергеевич пушкин 
  
 евгений онегин 
 роман в стихах 
  
 не мысля гордый свет забавить , 
 вниманье дружбы возлюбя , 
 хотел бы я тебе представить 
 залог достойнее тебя , 
 достойнее души прекрасной , 
 святой исполненной мечты , 
 поэзии живой и ясной , 
 высоких дум и простоты 
 но так и быть  рукой пристрастной 
 прими собранье пестрых глав , 
 полусмешных , полупечальных , 
 простонародных , идеальных , 
 небрежный плод моих забав , 
 бессонниц , легких вдохновений , 
 незрелых и увядших лет , 

ума холодных наблюдений 
 и сердца горестных замет . 
  
 глава первая 
  
 и жить торопится и чувствовать спешит . 
 кн . вяземский . 
  
 i 
  
 мой дядя самых честных правил , 
 когда не в шутку занемог , 
 он уважать себя заставил 
 и лучше выдумать не мог . 
 его пример другим наука 
 но , боже мой , какая скука 
 с больным сидеть и день и ночь , 
 не отходя ни шагу прочь 
 какое низкое коварство 
 полуживого забавлять , 
 ему подушки поправлять
, 
 печально подносить лека

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'александр сергеевич пушкин \n  \n евгений онегин \n роман в стихах \n  \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты \n но так и быть  рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n небрежный плод моих забав , \n бессонниц , легких вдохновений , \n незрелых и увядших лет ,'
Target data: 'сергеевич пушкин \n  \n евгений онегин \n роман в стихах \n  \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты \n но так и быть  рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n неб

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,   # tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
            vocab_size=len(vocab),
            embedding_dim=embedding_dim,
            rnn_units=rnn_units,
            batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 8566) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           2192896   
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 8566)          8780150   
                                                                 
Total params: 16220022 (61.87 MB)
Trainable params: 16220022 (61.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
print("Input: \n", repr(" ".join(idx2word[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices ])))

Input: 
 'мне делать с ней \n всем наотрез одно и то же \n нейду . и все грустит она , \n да бродит по лесам одна . \n  \n xxvi \n  \n не влюблена ль она  в кого же \n буянов сватался отказ . \n ивану петушкову  тоже . \n гусар пыхтин гостил у нас \n уж как он танею прельщался , \n как мелким бесом рассыпался \n я думала пойдет авось \n куда и снова дело врозь .  \n что ж , матушка за чем же стало \n в москву , на ярманку невест \n'

Next Char Predictions: 
 'механик музами поздной предо никем толкователь поздно иным легкомыслия длинной проказам молчалив мостики des томила заморозил чуждался уборе осенняя боясь читал страстное лай роман пролить петрович страсть письмах херувим веках гостиной умолк памяти любимый недвижим красна песен земли основанный стыда приветствий кибитка постели слышно блещет вся лились моих владений легкий золотым толкует греметь доверчивость листья кием зимняя гений охота сельский второй глупостей эти благородство гребут будущий весело басен пела ежегодно просту

### Обучение модели

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 8566)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.0553665


In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS = 100

In [25]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
5/5 [==============================] - 4s 317ms/step - loss: 8.3612
Epoch 2/100
5/5 [==============================] - 2s 296ms/step - loss: 6.4789
Epoch 3/100
5/5 [==============================] - 3s 730ms/step - loss: 6.3747
Epoch 4/100
5/5 [==============================] - 3s 495ms/step - loss: 6.2272
Epoch 5/100
5/5 [==============================] - 2s 416ms/step - loss: 6.1685
Epoch 6/100
5/5 [==============================] - 1s 300ms/step - loss: 6.1364
Epoch 7/100
5/5 [==============================] - 3s 641ms/step - loss: 6.1118
Epoch 8/100
5/5 [==============================] - 1s 290ms/step - loss: 6.0929
Epoch 9/100
5/5 [==============================] - 2s 312ms/step - loss: 6.0729
Epoch 10/100
5/5 [==============================] - 3s 691ms/step - loss: 6.0437
Epoch 11/100
5/5 [==============================] - 4s 938ms/step - loss: 6.0113
Epoch 12/100
5/5 [==============================] - 3s 591ms/step - loss: 5.9595
Epoch 13/100
5/5 [===================

### Генерация текста

In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            2192896   
                                                                 
 lstm_1 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_1 (Dense)             (1, None, 8566)           8780150   
                                                                 
Total params: 16220022 (61.87 MB)
Trainable params: 16220022 (61.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = []

    words = start_string.lower().strip().split(" ")
    for word in words:
        point, comma = False, False
        if "," in word:
            comma = True
        if "." in word:
            point = True
        word = ''.join([ch for ch in list(word) if ch not in string.punctuation])
        input_eval.append(word2vec[word])
        if comma:
            input_eval.append(word2vec[","])
        if point:
            input_eval.append(word2vec["."])
    # input_eval = [word2vec[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        # print(predicted_id)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [30]:
start_string = u"Уж небо осенью дышало, \n Уж реже солнышко блистало, \n Короче становился день,"

print(generate_text(model, start_string=start_string))

Уж небо осенью дышало, 
 Уж реже солнышко блистало, 
 Короче становился день,
 и прошлой наконец 
 в деревне , ни балов , ни поэт , 
 ни общежития искусством 
  
 v 
  
 но ей . какое взор 
 в сем случае совсем неправ . 
 что с ним в каком в ладони  
  
 xiii 
  
 но , сплин мой страждущая творенье 
 завез в них реке 
 и в двадцать своей . 
  
 x 
  
 и вот ввели в ней . 
  
 xxxii 
  
 она мчались в воздухе нагретом 
 татьяна 
 не на почтовых , 
 ни остротою , ни тоской , 
 ни остротою , ни в очах , 
 обнявшись , плакали оне . 
 и деву этот лета 
 я должна не находит 
 в деревне , ни в розах , 
 ни остротою , ни умом 
 в деревне , на мельницу . 
  
 i 
  
 и наконец , преданный безделью , 
 и , скачут и забор . 
  
 xxv 
  
 и вот ввели в тишине 
 в общих мнений , лугами 
 и вынулось колечко ей 
 осталася длинный страшно в залог . 
  
 iii 
  
 татьяна точно умиленным .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .